<a href="https://colab.research.google.com/github/erturkkadir/kurtahmini/blob/master/TransformerKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

raw_data = pd.read_csv('./datasets/ue128.csv', header=None, names=["i", "t", "y"])